### 다이내믹 프로그래밍 : 정책반복 알고리즘

#### 필요한 패키지를 불러온다.

In [ ]:
import numpy as np
from numpy.linalg import inv

#### 보상, 전이확률, 상태값, 감마 설정.

In [ ]:
R = np.array([[-0.02, -0.02, -0.02, -0.02],[-0.02, 0.0, -0.02, -1.0],[-0.02,-0.02,-0.02,1]])
P = np.zeros((3,4,4))
gamma = 0.9

In [ ]:
# =============
P[0,0,0] = 0.5
P[0,0,1] = 0.0
P[0,0,2] = 0.0
P[0,0,3] = 0.5
# -------------
P[0,1,0] = 0.5
P[0,1,1] = 0.0
P[0,1,2] = 0.5
P[0,1,3] = 0.0
# -------------
P[0,2,0] = 1.0/3.0
P[0,2,1] = 0.0
P[0,2,2] = 1.0/3.0
P[0,2,3] = 1.0/3.0
# -------------
P[0,3,0] = 0.0
P[0,3,1] = 0.0
P[0,3,2] = 0.5
P[0,3,3] = 0.5
# =============
P[1,0,0] = 0.0
P[1,0,1] = 0.5
P[1,0,2] = 0.0
P[1,0,3] = 0.5
# -------------
P[1,2,0] = 1.0/3.0
P[1,2,1] = 1.0/3.0
P[1,2,2] = 0.0
P[1,2,3] = 1.0/3.0
# -------------
P[1,3,0] = 0.0                 # 함정.
P[1,3,1] = 0.0                 #
P[1,3,2] = 0.0                 #
P[1,3,3] = 0.0                 #
# =============
P[2,0,0] = 0.5
P[2,0,1] = 0.5
P[2,0,2] = 0.0
P[2,0,3] = 0.0
# -------------
P[2,1,0] = 0.5
P[2,1,1] = 0.0
P[2,1,2] = 0.5
P[2,1,3] = 0.0
# -------------
P[2,2,0] = 1.0/3.0
P[2,2,1] = 1.0/3.0
P[2,2,2] = 1.0/3.0
P[2,2,3] = 0.0
# -------------
P[2,3,0] = 0.0                     # 목적지.
P[2,3,1] = 0.0                     #
P[2,3,2] = 0.0                     #
P[2,3,3] = 0.0                     #

#### 정책반복 알고리즘.

In [ ]:
# i_dir = 0~3으로 {오른쪽, 위, 왼쪽, 아래} 방향을 나타냄.
def V_val(y,x,i_dir):
    if (i_dir == 0 and x  < 3):
        return V[y,x + 1]
    elif (i_dir == 1 and y > 0):
        return V[y - 1, x]
    elif (i_dir == 2 and x > 0):
        return V[y, x - 1]
    elif (i_dir == 3 and y < 2):
        return V[y + 1, x]
    return 0.0

In [ ]:
# i_dir = 0~3에 의해서 {오른쪽, 위, 왼쪽, 아래} 방향으로 욺직임 출력.
def my_move(y,x,i_dir):
    if (i_dir == 0 and x  < 3):
        return (0,1)
    elif (i_dir == 1 and y > 0):
        return (-1,0)
    elif (i_dir == 2 and x > 0):
        return (0,-1)
    elif (i_dir == 3 and y < 2):
        return (1,0)
    return (0,0)

In [ ]:
# s => 좌표 딕셔너리.
s2coord = {0:(0,0), 1:(0,1), 2:(0,2), 3:(0,3), 4:(1,0), 5:(1,2), 6:(1,3), 7:(2,0), 8:(2,1), 9:(2,2), 10:(2,3)}
# 좌표 => s 함수.
def coord2s(i,j):
    for s in range(11):
        y, x = s2coord[s]
        if (y == i and x ==j):
            return s

In [ ]:
# 0으로 초기화.
V = np.zeros((3,4))   
# 임의의 정책 초기화. 이동 방향의 리스트.
pol = [0,0,0,3,3,0,0,0,0,0,0]

In [ ]:
# 선형 대수학 방법으로 V 행렬를 구한다.
def getV():
    mat = np.zeros((11,11))
    A = np.zeros((11,1))
    for s in range(11):
        i, j = s2coord[s]
        A[s,0] = R[i,j] 
        mat[s,s] = 1.0
        dy,dx = my_move(i,j,pol[s])
        ip, jp = i, j 
        ip += dy
        jp += dx
        sp = coord2s(ip,jp)
        mat[s,sp] += -gamma*P[i,j,pol[s]]
    # 선형 방정식 풀이.
    mat_inv = inv(mat)
    V_sol = np.dot(mat_inv, A)               
    for s in range(11):                         # V 행렬 교체.
        i, j = s2coord[s]
        V[i, j] = V_sol[s] 

In [ ]:
# 반복 횟수.
n_iter = 100
for _ in range(n_iter):
    getV()
    # policy 갱신
    pol = []
    for s in range(11):
        i, j = s2coord[s]
        temp = []
        for i_dir in range(4):
            if P[i,j,i_dir] > 0.0:
                temp +=  [P[i,j,i_dir]*V_val(i,j,i_dir)]
            else:
                temp += [-999]
        pol += [np.argmax(temp)]

In [ ]:
# 최종 상태 가치 출력.
print(np.round(V,3))

#### 최적의 정책을 추출한다.

In [ ]:
i=0             # 초기 세로 좌표.
j=0             # 초기 가로 좌표.
n_iter2 = 10

In [ ]:
policy = [(i,j)]
for _ in range(n_iter2):
    temp = []
    for i_dir in range(4):
        if P[i,j,i_dir] > 0.0:
            temp +=  [gamma*P[i,j,i_dir]*V_val(i,j,i_dir)]
        else:
            temp += [-999]
    dy,dx = my_move(i,j,np.argmax(temp))
    i += dy
    j += dx
    policy += [(i,j)]
    if (i == 2 and j == 3):
        break

In [ ]:
policy